# 洋流資料分析
https://pypi.python.org/pypi/folium  
https://github.com/python-visualization/folium  
https://folium.readthedocs.io/en/latest/  
https://ocefpaf.github.io/python4oceanographers/blog/2015/11/16/folium_quiver/  
http://www.digital-geography.com/python-and-webmaps-folium/#.V6w07Pl96T8

In [ ]:
%pylab inline
import pandas as pd
import folium
import mplleaflet
# from pprint import pprint

In [ ]:
def read_data_file(file):

    with open(file) as f:
        df = pd.read_csv(f,  
                         delim_whitespace = True,
                         comment = '%', 
                         header = None
                        )
    
    df.index.name = 'No.' 

    columns = ['Longitude', 'Latitude', 'U comp', 'V comp', 'VectorFlag', 'U StdDev', 'V StdDev', 
               'Covariance', 'StdDev', 'X Distance', 'Y Distance', 'Range', 'Bearing', 'Velocity', 'Direction']    
    site_contributors_count = 16
    columns.extend(['Site Contributor {0}'.format(i) for i in range(1, site_contributors_count + 1)])
    df.columns = columns
    
    return df

In [ ]:
def do_mplleaflet(lons, lats, u, v, 
                  sub = 1, scale = 1000,
                  color = 'deepskyblue',
                  alpha = 0.8, 
                  file = 'mplleaflet_map.html',
                  tiles = 'esri_aerial'):
    
    ax.quiver(lons[::sub], lats[::sub], u[::sub], v[::sub], 
              color = color, alpha = alpha, scale = scale)

    gj = mplleaflet.fig_to_geojson(fig = fig) 
    mplleaflet.save_html(fig = fig, fileobj = file, tiles = tiles)
#     mplleaflet.display(fig = fig, tiles = 'esri_aerial')
    
    return gj

In [ ]:
def get_quiver_marker_layer(geojson):
    feature_group = folium.FeatureGroup(name='quiver')

    for feature in geojson['features']:
        if feature['geometry']['type'] == 'Point':
            lon, lat = feature['geometry']['coordinates']
            div = feature['properties']['html']

            icon_anchor = (feature['properties']['anchor_x'],
                           feature['properties']['anchor_y'])

            icon = folium.features.DivIcon(html = div, icon_anchor = icon_anchor)
            marker = folium.Marker([lat, lon], icon = icon)
            feature_group.add_children(marker)
        else:
            msg = "Unexpected geometry {}".format
            raise ValueError(msg(feature['geometry']))

    return feature_group

In [ ]:
def get_WMS_tile_layer(url = 'http://gmrt.marine-geo.org/cgi-bin/mapserv?map=/public/mgg/web/gmrt.marine-geo.org/htdocs/services/map/wms_merc.map'):   
    
    wms = folium.features.WmsTileLayer(url, name='GMRT',
                                       format='image/png',
                                       layers='topo',
                                       transparent=True)
    
    feature_group = folium.FeatureGroup(name='Topo')
    feature_group.add_children(wms)
    
    return feature_group

In [ ]:
if __name__ == '__main__':
    
    file = 'TOTL_TORO_2015W01.tuv'
    df = read_data_file(file)
    output_file = 'folium_map.html'
    
    fig, ax = plt.subplots()
    gj = do_mplleaflet(df['Longitude'], df['Latitude'], df['U comp'], df['V comp'])
    gj = do_mplleaflet(df['Longitude'], df['Latitude'], df['U comp'], df['V comp'], color = 'black', file = output_file)
    
    quiver_layer = get_quiver_marker_layer(gj)    
    WMS_tile_layer = get_WMS_tile_layer()
    
    mapOBJ = folium.Map(location = [df.Latitude.mean(), df.Longitude.mean()], tiles = "OpenStreetMap", zoom_start = 7)

    mapOBJ.add_children(WMS_tile_layer)
    mapOBJ.add_children(quiver_layer)
    mapOBJ.add_children(folium.map.LayerControl())

    mapOBJ.save(output_file)    